# RANSACRegression (P,A,D coordinates to LMA Features)

In [3]:
# Current best:
#datasets/pad/Fs_B_S_DANCE_WALK_0.5sec.csv
"""
NOTE TO SELF: DON'T FREAKING USE THE STANDARDIZED/NORMALIZED DATASETS HERE, THEY WON'T WORK. YOU'RE STANDARDIZING THE OUTPUT DUMB DUMB!
                
"""

'\n\n                \n'

## Imports

In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib

from sklearn.linear_model import RANSACRegressor

import xgboost as xgb

xgb.set_config(verbosity=0)

## Load Data

In [4]:
dataset = pd.read_csv('datasets/Fs_B_O_DANCE_WALK_0.5sec.csv')

### Split into Test and Train data

In [5]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 12643
No Test Samples: 3161


### Split Features from Targets

In [6]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [7]:
train_X = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_X = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)

In [8]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [9]:
train_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
11682,0.1,-0.70,-0.20
10283,0.1,-0.70,-0.20
7174,-0.6,-0.30,-0.30
9892,0.9,-0.25,0.65
8682,-0.5,0.60,0.90


In [10]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Define Model

In [12]:


models = []

for i in range(len(train_y_sets)):
    models.append(
        RANSACRegressor()
    )

## Train Model

In [13]:
for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    models[i].fit(train_X, train_y_sets[i])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Test Model

### Training Score

In [14]:
for i in range(len(models)):
    score = models[i].score(train_X, train_y_sets[i])
    print(train_y_sets[i].columns[0])
    print("Training score: ", score)

    print()

max_hand_distance
Training score:  -0.16849518584499257

avg_l_hand_hip_distance
Training score:  -0.19174944522035897

avg_r_hand_hip_distance
Training score:  -0.22588591886152698

max_stride_length
Training score:  0.028616082836382173

avg_l_hand_chest_distance
Training score:  0.012606351270674665

avg_r_hand_chest_distance
Training score:  0.02185942837252086

avg_l_elbow_hip_distance
Training score:  -0.13103093997066462

avg_r_elbow_hip_distance
Training score:  -0.009804513421440841

avg_chest_pelvis_distance
Training score:  -0.23035646153237943

avg_neck_chest_distance
Training score:  -0.422538980436709

avg_neck_rotation_w
Training score:  -0.12311715823662595

avg_neck_rotation_x
Training score:  -0.31013267634302544

avg_neck_rotation_y
Training score:  -0.03638257532643374

avg_neck_rotation_z
Training score:  -0.3549067623642119

avg_total_body_volume
Training score:  -0.26433270261663466

avg_triangle_area_hands_neck
Training score:  -0.06679467004677297

avg_triangle

### Test Set MAE & MSE

In [17]:
for i in range(len(models)):
    pred_y = models[i].predict(test_X)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.5f" % mse)
    print("MAE: %.5f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.13753
MAE: 0.27936
Example:  0.4985718499677543

avg_l_hand_hip_distance
MSE: 0.05081
MAE: 0.15651
Example:  0.3778616762660692

avg_r_hand_hip_distance
MSE: 0.05036
MAE: 0.15958
Example:  0.3891095324253554

max_stride_length
MSE: 0.03273
MAE: 0.14250
Example:  0.7039830246660411

avg_l_hand_chest_distance
MSE: 0.01387
MAE: 0.08585
Example:  0.3193599887791773

avg_r_hand_chest_distance
MSE: 0.01405
MAE: 0.08857
Example:  0.4329107175084802

avg_l_elbow_hip_distance
MSE: 0.01103
MAE: 0.07521
Example:  0.2468938955558369

avg_r_elbow_hip_distance
MSE: 0.00945
MAE: 0.07309
Example:  0.3229376131922172

avg_chest_pelvis_distance
MSE: 0.00000
MAE: 0.00000
Example:  0.2861510000520013

avg_neck_chest_distance
MSE: 0.00002
MAE: 0.00291
Example:  0.2747842906654131

avg_neck_rotation_w
MSE: 0.02211
MAE: 0.11085
Example:  0.1252765615013744

avg_neck_rotation_x
MSE: 0.29184
MAE: 0.42420
Example:  0.1942804285214837

avg_neck_rotation_y
MSE: 0.02673
MAE: 0.11683
Exampl

### Prediction Examples

In [20]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [32]:
index = 1

real_coordinates = np.asarray([test_X.iloc[index]])

real_coordinates = pd.DataFrame(real_coordinates, columns=[
            "EMOTION_P","EMOTION_A","EMOTION_D"])


generated_features = []
for i in range(len(models)):
    generated_features.append(models[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates)[0])
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [ 0.9  -0.25  0.65]
Predicted: [0.23675478, 0.08128769, 0.39573506]


## Save Models

In [118]:
# save to JSON
model_p.save_model("models/l2p_dance_model.json")
model_a.save_model("models/l2a_dance_model.json")
model_d.save_model("models/l2d_dance_model.json")

## Load Models

In [856]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("models/l2p_dance_model_v2.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("models/l2a_dance_model_v2.json")

# Hyperparameter Grid Search

In [2]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


import xgboost as xgb

xgb.set_config(verbosity=2)

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import joblib

## Load Data

In [3]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_KIN_0.5sec.csv')

### Split into Test and Train data

In [4]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 39539
No Test Samples: 9885


### Split Features from Targets

In [5]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [6]:
train_X = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_X = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)

In [7]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [8]:
train_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
7012,-0.85,-0.10,-0.80
19814,0.90,-0.25,0.65
423,-0.40,0.25,-0.10
45807,-0.50,0.60,0.90
28216,-0.35,0.70,-0.80


In [9]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_neck_rotation_w
avg_neck_rotation_x
avg_neck_rotation_y
avg_neck_rotation_z
avg_total_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## RandomSearchCV

In [10]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

n_iter = 200

In [11]:
models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=1500, 
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )
    )

In [ ]:
# Pleasure
# run randomized search

random_searches = []

for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    
    kfold = KFold(n_splits=3, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_X, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

max_hand_distance
GridSearchCV took 1006.77 seconds parameter settings.

avg_l_hand_hip_distance
GridSearchCV took 1297.47 seconds parameter settings.

avg_r_hand_hip_distance


### Results

In [20]:
best_regressors = []

for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    best_regressor = random_searches[i].best_estimator_

    print(best_regressor.get_params())
    best_regressors.append(best_regressor)
    print()

max_hand_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0.01, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.100000001, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 11, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 1500, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.75, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.1, 'alpha': 0.0}

avg_l_hand_hip_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.75, 'enable_categorical': False, 'gamma': 0.01, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate

In [17]:
for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_X)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.07
MAE: 0.21
Example:  0.3589408342450211

avg_l_hand_hip_distance
MSE: 0.03
MAE: 0.12
Example:  0.2153851628650174

avg_r_hand_hip_distance
MSE: 0.03
MAE: 0.13
Example:  0.2099208139313444

max_stride_length
MSE: 0.03
MAE: 0.13
Example:  0.3014104931041282

avg_l_hand_chest_distance
MSE: 0.01
MAE: 0.09
Example:  0.437371900332585

avg_r_hand_chest_distance
MSE: 0.01
MAE: 0.09
Example:  0.4379690956765778

avg_l_elbow_hip_distance
MSE: 0.01
MAE: 0.06
Example:  0.315309309619572

avg_r_elbow_hip_distance
MSE: 0.01
MAE: 0.06
Example:  0.315764635359871

avg_chest_pelvis_distance
MSE: 0.00
MAE: 0.00
Example:  0.2861510014037163

avg_neck_chest_distance
MSE: 0.00
MAE: 0.00
Example:  0.2785828270734098

avg_neck_rotation_w
MSE: 0.01
MAE: 0.07
Example:  -0.0303669968027351

avg_neck_rotation_x
MSE: 0.10
MAE: 0.21
Example:  0.9944506264704334

avg_neck_rotation_y
MSE: 0.02
MAE: 0.10
Example:  -0.0616060786573984

avg_neck_rotation_z
MSE: 0.06
MAE: 0.13
Example:  -0.04

In [18]:
# save to JSON
for i in range(len(best_regressors)):
    best_regressors[i].save_model("models/0.5_sec/" + test_y_sets[i].columns[0] +".json")

### Predictions

In [46]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [47]:
index = 10

real_coordinates = np.asarray([test_X.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(best_regressors[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=[
            "max_hand_distance",
            "avg_l_hand_hip_distance",
            "avg_r_hand_hip_distance",
            "max_stride_length",
            "avg_l_hand_chest_distance",
            "avg_r_hand_chest_distance",
            "avg_l_elbow_hip_distance",
            "avg_r_elbow_hip_distance",
            "avg_chest_pelvis_distance",
            "avg_neck_chest_distance",
            "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
            "avg_total_body_volume",
            "avg_triangle_area_hands_neck",
            "avg_triangle_area_feet_hips",
          
            "l_hand_speed",
            "r_hand_speed",
            "l_foot_speed",
            "r_foot_speed",
            "neck_speed",
          
            "l_hand_acceleration_magnitude",
            "r_hand_acceleration_magnitude",
            "l_foot_acceleration_magnitude",
            "r_foot_acceleration_magnitude",
            "neck_acceleration_magnitude",
         ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [[ 0.9  -0.25  0.65]]
Predicted: [0.45704892, 0.14212963, 0.21817927]
